In [111]:
import pandas as pd
import numpy as np
import matplotlib as plt
import openpyxl
from datetime import datetime

Feature 1: read in 2 datasets. I pulled the Annual visitation numbers for Mammoth Cave as my first dataset. This can be found at irma.nps.gov - full link included in ReadMe.  

In [112]:
mammoth_df = pd.read_excel('park_visitation_1904_current.xlsx')
mammoth_df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Mammoth Cave NP,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,Year,Recreation Visits
3,NaN,NaN,1936,57775
4,NaN,NaN,1937,75434


Okay the formatting is weird here, but I'll fix that later. Now to import the second dataset, gas prices. This is an excel file with two separate sheets and the data I want is on sheet 2, titled "Data 1". 

Feature 1, dataset 2: Annual gas prices.

In [113]:
gas_df = pd.read_excel('gas_data_1994_current.xls', 'Data 1')
gas_df.head()

,Back to Contents,Data 1: U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)
0,Sourcekey,EMM_EPM0_PTE_NUS_DPG
1,Date,U.S. All Grades All Formulations Retail Gasoli...
2,1993-06-30 00:00:00,NaN
3,1994-06-30 00:00:00,1.078
4,1995-06-30 00:00:00,1.158


This dataset only goes back to 1994, so I guess I need to exclude annual visitor info before that date. Gotta name the columns and then filter by year.

Feature 2: data cleaning. 

In [114]:
mammoth_df_1994 = mammoth_df.rename(columns={'Unnamed: 0': 'Placeholder', 'Unnamed: 1': 'NP', 'Unnamed: 2': 'Year', 'Unnamed: 3': 'Recreation Visits'})
mammoth_df_1994.columns.tolist()



['Placeholder', 'NP', 'Year', 'Recreation Visits']

The first two columns are kinda pointless. Let's drop those. 

In [115]:
mammoth_df_1994 = mammoth_df_1994.drop(columns=['Placeholder', 'NP'])

In [116]:
mammoth_df_1994.drop([0,1,2,90], axis=0, inplace=True)
mammoth_df_1994.head()

,Year,Recreation Visits
3,1936,57775
4,1937,75434
5,1938,120692
6,1939,116516
7,1940,117751


In [117]:
print(mammoth_df_1994)

    Year Recreation Visits
3   1936             57775
4   1937             75434
5   1938            120692
6   1939            116516
7   1940            117751
..   ...               ...
85  2018            533206
86  2019            551590
87  2020            290392
88  2021            515774
89  2022            663147

[87 rows x 2 columns]


In [118]:
mammoth_df_1994.describe()

,Year,Recreation Visits
count,87,87
unique,87,87
top,1936,57775
freq,1,1


In [119]:
mammoth_df_1994.dtypes

Year                 object
Recreation Visits    object
dtype: object

Both columns are objects and I will need them to be numeric values to do the analyses I want to run. Time to convert!

In [120]:
mammoth_df_1994.replace(',','', regex=True, inplace=True)
mammoth_df_1994.head()

,Year,Recreation Visits
3,1936,57775
4,1937,75434
5,1938,120692
6,1939,116516
7,1940,117751


In [121]:
mammoth_df_1994.dtypes

Year                 int64
Recreation Visits    int64
dtype: object

Now I need to filter all the visitor data so I can compare it to the gas prices. Gas prices don't start until 1994, so I will drop everything prior in the visitor set. 

In [122]:
mammoth_filter = mammoth_df_1994[mammoth_df_1994.Year >= 1994]
mammoth_filter.head()


,Year,Recreation Visits
61,1994,2009935
62,1995,1935709
63,1996,1896829
64,1997,1997658
65,1998,2113992


Back to the gas info. Time to clean it a little. The first 3 rows aren't needed. 

In [123]:
gas_df.drop([0,1,2], axis=0, inplace=True)
gas_df.head()

,Back to Contents,Data 1: U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)
3,1994-06-30 00:00:00,1.078
4,1995-06-30 00:00:00,1.158
5,1996-06-30 00:00:00,1.245
6,1997-06-30 00:00:00,1.244
7,1998-06-30 00:00:00,1.072


The first column is coming up with the name "Back to Contents"? Okay. Let's rename that to year. And that second column name is a doozy. That will get changed too. 

In [124]:
gas_df.rename(columns={gas_df.columns[0]: 'year', gas_df.columns[1]: 'price'}, inplace=True)
print(gas_df.columns)

Index(['year', 'price'], dtype='object')


The year column needs to be drilled down to just the year so I can merge the two dataframes together. 